# $$\text{Modelo de Predicción}$$

In [1]:
import pandas as pd
import plotly.express as px

In [2]:
bike_train = pd.read_excel('../bike_train.xlsx')
bike_train.head(2)

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40


- Lo que podemos suponer a primera vista, no usaremos la columna dteday ya que no queremos que el modelo dependa de una fecha que nunca es igual al anterior o al del dia siguiente. Lo que si podemos decir es que esta fecha ha sido dividida en columnas más independientes como la estación del año (season), el mes del año en el que ocurre el evento (mnth), la hora del registro (hr), la semana en la que ocurre (weekday). Que son eventos que se repiten siempre.

- También podemos prescindir de la columna instant, ya que es solo un identificador para llevar en orden los datos que se registran.

- Finalmente, tampoco usaremos las columnas "casual" y "registered", ya que queremos calcular la cantidad total de registrados, independientemente si son clientes vips de la empresa o solo toman el servicio cuando lo necesitan.

In [7]:
#Nuestra variable de predicción es la cantidad total de regsitrados (cnt), y veremos si existen alguna relación directa
# o inderecta entre las columnas = [season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed]

new_bike_train = bike_train.drop(['instant','dteday','casual','registered'], axis=1)
new_bike_train.head(2)

,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,16
1,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,40


In [5]:
fig = px.scatter_matrix(bike_train, dimensions=[
            # "season", "yr", "mnth",
            "casual","registered","cnt",
            # "hr","holiday","weekday","workingday","weathersit"
            "temp","atemp","hum","windspeed"])
fig.show()